In [68]:
from music21 import *
from typing import Optional, Dict, Any, List

In [12]:
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = r'C:\Program Files\MuseScore 3\bin\MuseScore3.exe'

In [54]:
path = 'bach_fugue_1-16_example.musicxml'

In [117]:
score = converter.parse(path)

In [56]:
score.show('midi')

In [116]:
score.hasVoices()

False

In [87]:
score = score.explode()

In [120]:
from music21 import converter, note, chord
import json

def musicxml_to_voice_json(xml_path):
    """
    Convert a MusicXML file into JSON format where each Part is treated as a single voice.
    """
    score = converter.parse(xml_path)
    score = score.explode()

    result = {
        "metadata": {},
        "voices": {}
    }

    # ---------- METADATA ----------
    ts = score.recurse().getElementsByClass("TimeSignature").first()
    ks = score.recurse().getElementsByClass("KeySignature").first()

    result["metadata"] = {
        "title": score.metadata.title if score.metadata else None,
        "composer": score.metadata.composer if score.metadata else None,
        "time_signature": ts.ratioString if ts else None,
        "key_signature": ks.sharps if ks else None,
        "divisions": score.highestTime
    }

    # ---------- PARTS AS VOICES ----------
    for part_idx, part in enumerate(score.parts, start=1):
        print(f"Processing Part {part_idx}: {part.id}")
        voice_key = f"P{part_idx}"
        events = []

        for el in part.flatten().notesAndRests:
            start_time = float(el.offset)
            duration = float(el.quarterLength)

            if el.isRest:
                events.append({
                    "t": start_time,
                    "dur": duration,
                    "rest": True
                })
            elif isinstance(el, note.Note):
                events.append({
                    "t": start_time,
                    "dur": duration,
                    "pitch": el.pitch.nameWithOctave
                })
            elif isinstance(el, chord.Chord):
                # Each pitch in the chord becomes a separate event at same time
                for p in el.pitches:
                    events.append({
                        "t": start_time,
                        "dur": duration,
                        "pitch": p.nameWithOctave
                    })

        result["voices"][voice_key] = sorted(events, key=lambda x: (x["t"], x.get("pitch", "")))

    return result


In [118]:
len(score.parts)

2

In [115]:
[(part_idx, part) for (part_idx, part) in enumerate(score.parts, start=1)]

[(1, <music21.stream.Part P1-Staff1-v0>),
 (2, <music21.stream.Part P1-Staff1-v1>),
 (3, <music21.stream.Part P1-Staff2-v0>)]

In [104]:
score.parts[1].show('midi')

In [121]:
data = musicxml_to_voice_json(path)

print(json.dumps(data, indent=2))

Processing Part 1: P1-Staff1-v0
Processing Part 2: P1-Staff1-v1
Processing Part 3: P1-Staff2-v0
{
  "metadata": {
    "title": null,
    "composer": null,
    "time_signature": "4/4",
    "key_signature": 7,
    "divisions": 64.0
  },
  "voices": {
    "P1": [
      {
        "t": 0.0,
        "dur": 2.0,
        "rest": true
      },
      {
        "t": 2.0,
        "dur": 0.5,
        "rest": true
      },
      {
        "t": 2.5,
        "dur": 0.5,
        "pitch": "G#4"
      },
      {
        "t": 3.0,
        "dur": 0.5,
        "pitch": "B#4"
      },
      {
        "t": 3.5,
        "dur": 0.5,
        "pitch": "G#4"
      },
      {
        "t": 4.0,
        "dur": 0.5,
        "pitch": "C#5"
      },
      {
        "t": 4.5,
        "dur": 0.5,
        "rest": true
      },
      {
        "t": 5.0,
        "dur": 0.5,
        "pitch": "B#4"
      },
      {
        "t": 5.5,
        "dur": 0.5,
        "rest": true
      },
      {
        "t": 6.0,
        "dur": 0.25

In [125]:
score.explode().parts[2].show('text')

{0.0} <music21.instrument.Instrument 'P1: Фортепіано: Фортепіано'>
{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.layout.SystemLayout>
    {0.0} <music21.layout.StaffLayout distance 65, staffNumber 2, staffSize None, staffLines None>
    {0.0} <music21.clef.BassClef>
    {0.0} <music21.key.KeySignature of 7 sharps>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Rest eighth>
    {0.5} <music21.note.Note C#>
    {1.0} <music21.note.Note E#>
    {1.5} <music21.note.Note C#>
    {2.0} <music21.note.Note G#>
    {2.5} <music21.note.Rest eighth>
    {3.0} <music21.note.Note F#>
    {3.5} <music21.note.Rest eighth>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note E#>
    {0.25} <music21.note.Note D#>
    {0.5} <music21.note.Note C#>
    {1.0} <music21.note.Note C#>
    {1.25} <music21.note.Note D#>
    {1.5} <music21.note.Note E#>
    {1.75} <music21.note.Note C#>
    {2.0} <music21.note.Note F#>
    {2.5} <music21.note.Rest eighth>

In [128]:
from music21 import note, stream, chord, duration

def get_tinynotation_octave(pitch):
    """
    Map music21 pitch to the octave notation in your rules.
    CC to BB = C1 to B2 (MIDI 24-47)
    C to B = C2 to B3 (MIDI 36-59) 
    c to b = C4 to B4 (MIDI 60-71)
    c' to b' = C5 to B5 (MIDI 72-83)
    """
    octave = pitch.octave
    step = pitch.step.lower()
    
    if octave <= 1:
        # CCC, CC range (C0-B1)
        return pitch.step.upper() * (3 - octave)
    elif octave == 2:
        # CC to BB (C2-B2)
        return pitch.step.upper() * 2
    elif octave == 3:
        # C to B (C3-B3)
        return pitch.step.upper()
    elif octave == 4:
        # c to b (C4-B4, middle C octave)
        return step
    else:
        # c' to b' and higher (C5+)
        return step + "'" * (octave - 4)

def voice_to_tinynotation_custom(part: stream.Part) -> str:
    """
    Converts a single music21 Part (voice) to custom tinynotation
    according to the user rules.
    """
    tn = []
    prev_duration = None
    
    for elem in part.recurse().notesAndRests:
        # Duration mapping
        dur_map = {
            'whole': '1',
            'half': '2',
            'quarter': '4',
            'eighth': '8',
            '16th': '16',
            '32nd': '32',
            '64th': '64'
        }
        dur_num = dur_map.get(elem.duration.type, '4')
        
        # Dot
        dot_str = '.' if elem.duration.dots > 0 else ''
        
        # Check if we need to specify duration
        full_duration = dur_num + dot_str
        if full_duration == prev_duration:
            duration_str = ''
        else:
            duration_str = full_duration
            prev_duration = full_duration
        
        # Tie
        tie_str = '~' if elem.tie and elem.tie.type == 'start' else ''
        
        if isinstance(elem, note.Rest):
            # Always specify duration for rests
            tn.append(f"r{dur_num}{dot_str}{tie_str}")
        elif isinstance(elem, note.Note):
            # Get base note name (just the letter)
            base_name = elem.pitch.step.lower()
            
            # Accidental
            acc = ''
            if elem.pitch.accidental:
                alter = elem.pitch.accidental.alter
                if alter == 1:
                    acc = '#'
                elif alter == -1:
                    acc = '-'
                elif alter == 0:
                    acc = 'n'
                elif alter == 2:
                    acc = '##'
                elif alter == -2:
                    acc = '--'
                
                # Editorial accidentals in parentheses
                if hasattr(elem.pitch.accidental, 'editorial') and elem.pitch.accidental.editorial:
                    acc = f"({acc})"
            
            # Octave designation
            octave_str = get_tinynotation_octave(elem.pitch)
            
            # The octave_str already contains the letter, so extract just the octave part
            # Find where the letter ends
            letter = elem.pitch.step.upper() if octave_str[0].isupper() else elem.pitch.step.lower()
            octave_suffix = octave_str[1:]  # Everything after the first letter
            
            tn.append(f"{base_name}{acc}{octave_suffix}{duration_str}{tie_str}")
    
    return " ".join(tn)

In [137]:
before = score.explode().parts[0]
after = converter.parse(
    voice_to_tinynotation_custom(before), 
    format='tinynotation'
)

print(before == after)

False


In [141]:
before.show('text')

{0.0} <music21.instrument.Instrument 'P1: Фортепіано: Фортепіано'>
{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.key.KeySignature of 7 sharps>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Rest half>
    {2.0} <music21.note.Rest eighth>
    {2.5} <music21.note.Note G#>
    {3.0} <music21.note.Note B#>
    {3.5} <music21.note.Note G#>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note C#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note B#>
    {1.5} <music21.note.Rest eighth>
    {2.0} <music21.note.Note A#>
    {2.25} <music21.note.Note G#>
    {2.5} <music21.note.Note F#>
    {3.0} <music21.note.Note F#>
    {3.25} <music21.note.Note G#>
    {3.5} <music21.note.Note A#>
    {3.75} <music21.note.Note F#>
{8.0} <music21.stream.Measure 3 offset=8.0>
    {0.0} <music21.note.Note G#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note C#>
    {1.5} <music21.note.

In [152]:
before_str = """
{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Rest half>
    {2.0} <music21.note.Rest eighth>
    {2.5} <music21.note.Note G#>
    {3.0} <music21.note.Note B#>
    {3.5} <music21.note.Note G#>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note C#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note B#>
    {1.5} <music21.note.Rest eighth>
    {2.0} <music21.note.Note A#>
    {2.25} <music21.note.Note G#>
    {2.5} <music21.note.Note F#>
    {3.0} <music21.note.Note F#>
    {3.25} <music21.note.Note G#>
    {3.5} <music21.note.Note A#>
    {3.75} <music21.note.Note F#>
{8.0} <music21.stream.Measure 3 offset=8.0>
    {0.0} <music21.note.Note G#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note C#>
    {1.5} <music21.note.Rest eighth>
    {2.0} <music21.note.Note C#>
    {2.5} <music21.note.Rest eighth>
    {3.0} <music21.note.Note B#>
    {3.5} <music21.note.Rest eighth>
{12.0} <music21.stream.Measure 4 offset=12.0>
    {0.0} <music21.note.Rest eighth>
    {0.5} <music21.note.Note C#>
    {1.0} <music21.note.Note E#>
    {1.5} <music21.note.Note C#>
    {2.0} <music21.note.Note G#>
    {2.5} <music21.note.Rest eighth>
    {3.0} <music21.note.Note F#>
    {3.5} <music21.note.Rest eighth>
{16.0} <music21.stream.Measure 5 offset=16.0>
    {0.0} <music21.note.Note E#>
    {0.25} <music21.note.Note D#>
    {0.5} <music21.note.Note C#>
    {1.0} <music21.note.Note C#>
    {1.25} <music21.note.Note D#>
    {1.5} <music21.note.Note E#>
    {1.75} <music21.note.Note C#>
    {2.0} <music21.note.Rest 16th>
    {2.25} <music21.note.Note D#>
    {2.5} <music21.note.Note F#>
    {2.75} <music21.note.Note D#>
    {3.0} <music21.note.Note G#>
    {3.5} <music21.note.Rest eighth>
{20.0} <music21.stream.Measure 6 offset=20.0>
    {0.0} <music21.note.Note F#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note E#>
    {1.75} <music21.note.Note E#>
    {2.0} <music21.note.Note F##>
    {2.5} <music21.note.Note G#>
    {3.0} <music21.note.Note G#>
    {3.25} <music21.note.Note G#>
    {3.5} <music21.note.Note F##>
{24.0} <music21.stream.Measure 7 offset=24.0>
    {0.0} <music21.note.Rest whole>
{28.0} <music21.stream.Measure 8 offset=28.0>
    {0.0} <music21.note.Rest whole>
{32.0} <music21.stream.Measure 9 offset=32.0>
    {0.0} <music21.note.Rest whole>
{36.0} <music21.stream.Measure 10 offset=36.0>
    {0.0} <music21.note.Rest whole>
{40.0} <music21.stream.Measure 11 offset=40.0>
    {0.0} <music21.note.Rest whole>
{44.0} <music21.stream.Measure 12 offset=44.0>
    {0.0} <music21.note.Rest whole>
{48.0} <music21.stream.Measure 13 offset=48.0>
    {0.0} <music21.note.Rest whole>
{52.0} <music21.stream.Measure 14 offset=52.0>
    {0.0} <music21.note.Rest whole>
{56.0} <music21.stream.Measure 15 offset=56.0>
    {0.0} <music21.note.Rest whole>
{60.0} <music21.stream.Measure 16 offset=60.0>
    {0.0} <music21.note.Rest whole>
    {4.0} <music21.bar.Barline type=final>
"""

In [140]:
after.show('text')

{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Rest half>
    {2.0} <music21.note.Rest eighth>
    {2.5} <music21.note.Note G#>
    {3.0} <music21.note.Note B#>
    {3.5} <music21.note.Note G#>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note C#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note B#>
    {1.5} <music21.note.Rest eighth>
    {2.0} <music21.note.Note A#>
    {2.25} <music21.note.Note G#>
    {2.5} <music21.note.Note F#>
    {3.0} <music21.note.Note F#>
    {3.25} <music21.note.Note G#>
    {3.5} <music21.note.Note A#>
    {3.75} <music21.note.Note F#>
{8.0} <music21.stream.Measure 3 offset=8.0>
    {0.0} <music21.note.Note G#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note C#>
    {1.5} <music21.note.Rest eighth>
    {2.0} <music21.note.Note C#>
    {2.5} <music21.note.Rest eighth>
    {3.0} <music21.note.Note B#>


In [153]:
after_str = """
{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Rest half>
    {2.0} <music21.note.Rest eighth>
    {2.5} <music21.note.Note G#>
    {3.0} <music21.note.Note B#>
    {3.5} <music21.note.Note G#>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note C#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note B#>
    {1.5} <music21.note.Rest eighth>
    {2.0} <music21.note.Note A#>
    {2.25} <music21.note.Note G#>
    {2.5} <music21.note.Note F#>
    {3.0} <music21.note.Note F#>
    {3.25} <music21.note.Note G#>
    {3.5} <music21.note.Note A#>
    {3.75} <music21.note.Note F#>
{8.0} <music21.stream.Measure 3 offset=8.0>
    {0.0} <music21.note.Note G#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note C#>
    {1.5} <music21.note.Rest eighth>
    {2.0} <music21.note.Note C#>
    {2.5} <music21.note.Rest eighth>
    {3.0} <music21.note.Note B#>
    {3.5} <music21.note.Rest eighth>
{12.0} <music21.stream.Measure 4 offset=12.0>
    {0.0} <music21.note.Rest eighth>
    {0.5} <music21.note.Note C#>
    {1.0} <music21.note.Note E#>
    {1.5} <music21.note.Note C#>
    {2.0} <music21.note.Note G#>
    {2.5} <music21.note.Rest eighth>
    {3.0} <music21.note.Note F#>
    {3.5} <music21.note.Rest eighth>
{16.0} <music21.stream.Measure 5 offset=16.0>
    {0.0} <music21.note.Note E#>
    {0.25} <music21.note.Note D#>
    {0.5} <music21.note.Note C#>
    {1.0} <music21.note.Note C#>
    {1.25} <music21.note.Note D#>
    {1.5} <music21.note.Note E#>
    {1.75} <music21.note.Note C#>
    {2.0} <music21.note.Rest 16th>
    {2.25} <music21.note.Note D#>
    {2.5} <music21.note.Note F#>
    {2.75} <music21.note.Note D#>
    {3.0} <music21.note.Note G#>
    {3.5} <music21.note.Rest eighth>
{20.0} <music21.stream.Measure 6 offset=20.0>
    {0.0} <music21.note.Note F#>
    {0.5} <music21.note.Rest eighth>
    {1.0} <music21.note.Note E#>
    {1.75} <music21.note.Note E#>
    {2.0} <music21.note.Note F##>
    {2.5} <music21.note.Note G#>
    {3.0} <music21.note.Note G#>
    {3.25} <music21.note.Note G#>
    {3.5} <music21.note.Note F##>
{24.0} <music21.stream.Measure 7 offset=24.0>
    {0.0} <music21.note.Rest whole>
{28.0} <music21.stream.Measure 8 offset=28.0>
    {0.0} <music21.note.Rest whole>
{32.0} <music21.stream.Measure 9 offset=32.0>
    {0.0} <music21.note.Rest whole>
{36.0} <music21.stream.Measure 10 offset=36.0>
    {0.0} <music21.note.Rest whole>
{40.0} <music21.stream.Measure 11 offset=40.0>
    {0.0} <music21.note.Rest whole>
{44.0} <music21.stream.Measure 12 offset=44.0>
    {0.0} <music21.note.Rest whole>
{48.0} <music21.stream.Measure 13 offset=48.0>
    {0.0} <music21.note.Rest whole>
{52.0} <music21.stream.Measure 14 offset=52.0>
    {0.0} <music21.note.Rest whole>
{56.0} <music21.stream.Measure 15 offset=56.0>
    {0.0} <music21.note.Rest whole>
{60.0} <music21.stream.Measure 16 offset=60.0>
    {0.0} <music21.note.Rest whole>
    {4.0} <music21.bar.Barline type=final>
"""

In [155]:
len(before_str.strip().split('\n')), len(after_str.strip().split('\n'))

(83, 83)

In [154]:
before_list = before_str.strip().split('\n')
after_list = after_str.strip().split('\n')

for i in range(min(len(before_list), len(after_list))):
    print(f"Before: {before_list[i]}")
    print(f"After:  {after_list[i]}")
    print()

Before: {0.0} <music21.stream.Measure 1 offset=0.0>
After:  {0.0} <music21.stream.Measure 1 offset=0.0>

Before:     {0.0} <music21.clef.TrebleClef>
After:      {0.0} <music21.clef.TrebleClef>

Before:     {0.0} <music21.meter.TimeSignature 4/4>
After:      {0.0} <music21.meter.TimeSignature 4/4>

Before:     {0.0} <music21.note.Rest half>
After:      {0.0} <music21.note.Rest half>

Before:     {2.0} <music21.note.Rest eighth>
After:      {2.0} <music21.note.Rest eighth>

Before:     {2.5} <music21.note.Note G#>
After:      {2.5} <music21.note.Note G#>

Before:     {3.0} <music21.note.Note B#>
After:      {3.0} <music21.note.Note B#>

Before:     {3.5} <music21.note.Note G#>
After:      {3.5} <music21.note.Note G#>

Before: {4.0} <music21.stream.Measure 2 offset=4.0>
After:  {4.0} <music21.stream.Measure 2 offset=4.0>

Before:     {0.0} <music21.note.Note C#>
After:      {0.0} <music21.note.Note C#>

Before:     {0.5} <music21.note.Rest eighth>
After:      {0.5} <music21.note.Rest eigh

In [156]:
before_str == after_str

True

In [171]:
tinyscore = ""
for id, voice in enumerate(score.explode().parts):
    tn_str = voice_to_tinynotation_custom(voice)
    tinyscore += f"V{id}: {tn_str}\n"
print(tinyscore)

V0: r2 r8 g# b# g# c#' r8 b# r8 a#16 g# f#8~ f#16 g# a# f# g#8 r8 c#' r8 c#' r8 b# r8 r8 c#' e#' c#' g#' r8 f#' r8 e#'16 d#' c#'8~ c#'16 d#' e#' c#' r16 d#' f#' d#' g#'8 r8 f#' r8 e#'8. e#'16 f##'8 g#'~ g#'16 g#' f##'8 r1 r1 r1 r1 r1 r1 r1 r1 r1 r1
V1: r1 r8 g# e# g# c# r8 d# r8 e#16 f# g#8~ g#16 f# e# g# d# e# f#8~ f#16 e# d# f# e#8 r8 r4 r8 g# b# g# c#' r8 bn r8 a# r8 r16 a# c##' a# d#'8 r8 r16 d#' c#'8~ c#'16 b#8 b#16 a#8. c#'16 r1
V2: r8 c# e# c# g# r8 f# r8 e#16 d# c#8~ c#16 d# e# c# f#8 r8 f#F r8 r8 c# a#A c# f#F a#A d#D g#G c#C r8 r8 c# b# c# g# r8 r8 c# e# c# f# r8 e# r8 r16 d# b#B d# g##G8 a#A d#D e#E c#C d#D r1 r1 r1 r1 r1 r1 r1 r1 r1 r1



In [160]:
SYSTEM_PROMPT = """
    You are a symbolic music analysis assistant trained in Western tonal theory and Eastern European art music.
   Your task is not to generate music, but to analyze short symbolic score excerpts and annotate high-level musical entities in a conservative, explainable manner.
     When uncertain, you must say so explicitly.
"""

In [ ]:
USER_PROMPT_TEMPLATE = """
I will give you a short symbolic music excerpt encoded in TinyNotation.

## Context
Composer: {COMPOSER_NAME}
National tradition: {NATIONALITY} (Ukrainian / Western)
Approx. date / style: {STYLE} (Romantic / Folk-influenced / Modernist / etc.)
Key: {KEY}
Time signature: {METER}
Length: {NUMBER} bars (usually 16)


## Task
Please annotate high-level musical entities in the excerpt below.
You must:

Identify entities conservatively (do NOT over-annotate)
Use bar numbers
Specify the voice(s) involved for MOTIF entities
Justify each annotation briefly in plain musical language
Explicitly mark uncertainty if present

## Allowed entity types (ONLY these)
THEME – primary melodic idea, recurring or structurally central
MOTIF – short recurring melodic/rhythmic figure
PHRASE – musically coherent unit ending with a cadence or pause
CADENCE – harmonic or melodic closure (PAC, IAC, HC, DC, modal, unclear)
SEQUENCE – repeated material transposed stepwise
MODAL_HINT – evidence of non-functional or folk/modal behavior


## Output format (STRICT)
Return a JSON array, where each item has:
{{
"entity_type": "...",
"start_bar": X,
"end_bar": Y,
"voices": ["V0","V1",...], // only required for MOTIF, omit or leave empty for other entities
"confidence": "high | medium | low",
"justification": "Brief musical explanation"
}}
If no clear instance of an entity exists, do NOT invent one.

## TinyNotation excerpt
{PASTE_TINYNOTATION_HERE}


## Final check (IMPORTANT)
After producing the annotations, briefly answer:
Which annotations are most uncertain, and why?
Is the excerpt clearly tonal, folk-modal, or ambiguous?
Keep this final reflection under 4 sentences.
"""

In [ ]:
type(score)

NoneType

In [176]:
prompt = SYSTEM_PROMPT + USER_PROMPT_TEMPLATE.format(
    COMPOSER_NAME="J.S. Bach",
    NATIONALITY="German",
    STYLE="Baroque",          
    KEY="C# Major",
    METER="4/4",
    NUMBER=16,
    PASTE_TINYNOTATION_HERE=tinyscore,
)

In [178]:
print(prompt)


    You are a symbolic music analysis assistant trained in Western tonal theory and Eastern European art music.
   Your task is not to generate music, but to analyze short symbolic score excerpts and annotate high-level musical entities in a conservative, explainable manner.
     When uncertain, you must say so explicitly.

I will give you a short symbolic music excerpt encoded in TinyNotation.

## Context
Composer: J.S. Bach
National tradition: German (Ukrainian / Western)
Approx. date / style: Baroque (Romantic / Folk-influenced / Modernist / etc.)
Key: C# Major
Time signature: 4/4
Length: 16 bars (usually 16)


## Task
Please annotate high-level musical entities in the excerpt below.
You must:
Identify entities conservatively (do NOT over-annotate)
Use bar numbers 
Justify each annotation briefly in plain musical language
Explicitly mark uncertainty if present

## Allowed entity types (ONLY these)
THEME – primary melodic idea, recurring or structurally central
MOTIF – short recurri